This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'D4xZ6GasrTtJcQSbQ3UL'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
start_date = '2017-01-01'
end_date = '2017-12-29'
response = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X-Carl-Zeiss-Meditec-AFX_X/data.json?start_date={}&end_date={}&api_key={}".format(start_date, end_date, API_KEY))

In [44]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response_dict = response.json()
data = response_dict['dataset_data']['data']
cols = response_dict['dataset_data']['column_names']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
print(cols)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [6]:
data[0]

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [33]:
def row_to_dict(row):
    return {
        'high': row[2],
        'low': row[3],
        'close': row[4],
        'max_daily_change': row[2] - row[3],
        'volume': row[6]
    }

In [34]:
df = { row[0]: row_to_dict(row) for row in data}

In [45]:
# df

In [36]:
max_close = ('', 0)
for k in df:
    close = df[k]['close']
    if close > max_close[1]:
        max_close = (k, close)
        
print(max_close)

('2017-12-13', 53.09)


In [37]:
min_close = ('', 80)
for k in df:
    close = df[k]['close']
    if close < min_close[1]:
        min_close = (k, close)
        
print(min_close)

('2017-01-23', 34.06)


In [38]:
max_daily_change = ('', 0)
for k in df:
    change = df[k]['max_daily_change']
    if change > max_daily_change[1]:
        max_daily_change = (k, change)
        
print(max_daily_change)

('2017-05-11', 2.8100000000000023)


In [39]:
max_d2d_change = ('', '', 0)
dates = sorted(df.keys(), reverse=False)
max_change = 0
yesterday = dates[0]
for date in dates[1:]:
    yesterday_data = df[yesterday]
    today_data = df[date]
    change = abs(today_data['close'] - yesterday_data['close'])
    if change > max_change:
        max_d2d_change = (yesterday, date, change)
    yesterday = date
    
print(max_d2d_change)

('2017-12-28', '2017-12-29', 0.1599999999999966)


In [43]:
volumes = [i['volume'] for i in df.values()]
avg_volume = sum(volumes) / len(volumes)
avg_volume

89124.33725490196